In [1]:
import pandas as pd

In [2]:
df_customers = pd.read_csv("https://dojo-git.s3.eu-west-3.amazonaws.com/cleaned/customers.csv")
df_products = pd.read_csv("https://dojo-git.s3.eu-west-3.amazonaws.com/cleaned/products.csv")
df_transactions = pd.read_csv("https://dojo-git.s3.eu-west-3.amazonaws.com/cleaned/Transactions.csv")

In [3]:
df_customers.head()

,client_id;sex;birth
0,c_4410;f;1967
1,c_7839;f;1975
2,c_1699;f;1984
3,c_5961;f;1962
4,c_5320;m;1943
